Erica Sturm  
Womanium 2024: Classiq Exercise 6.12  

## Installation and import statements
Begin by installing Classiq and importing all required Python modules. Also connect/authenticate with the Classiq backend.

In [2]:
# Install Classiq
%pip install -U classiq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.9/387.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.

In [3]:
# Now, import statements. I don't know if we need numpy or matplotlib, but they're good to have on hand.
import classiq
from classiq import *
import numpy as np
import matplotlib.pyplot as plt

# And authenticate the Classiq backend
classiq.authenticate()

Your user code: RHBD-RXRJ
If a browser doesn't automatically open, please visit this URL from any trusted device: https://auth.classiq.io/activate?user_code=RHBD-RXRJ


## It's code time
This is where the assignment will "happen."  
Begin with an assignment for a global variable `SIZE` that will dictate the number of qubits required. In this assignment, we have been asked to execute the quantum random walk (QRW) on a linear graph of 16 nodes. Therefore, we will use `SIZE=4` qubits since $4^2=16$.

In [4]:
# Define global variable SIZE.
SIZE = 4

Let's define several helper functions for later in later cells. These are taken directly from the example notebook, but I have explained each function's purpose in comments for each.

In [5]:
"""
This function takes a single qubit 'x' that is presumably in the |0> state and places it in the |-> state.
This is part of initializing the phase kickback component of the algorithm, as |-> has a phase.
"""
@qfunc
def prepare_minus(x: QBit):
  X(x)
  H(x)

#-----------------------------------------------------------------------------------------------------------

"""
This function takes two qubit number registers x and aux. It will execute some boolean comparisons and
return the result (which will be either 0 or 1).

We begin with the part x!=0.
If x is 0, the overall result is False (ie 0). If x is not 0, then the overall result is True (ie 1).

Then, that previous result (of x!=0) and the aux qubit number register are compared using XOR.
If the values do not match, then aux is set to 1. If they do match, then aux is set to 0.
The final value of aux is then returned.
"""
@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
  aux^=(x!=0)

#-----------------------------------------------------------------------------------------------------------

"""
This subroutine creates the primitive code block for creating the phase kickback oracle.
First, set the auxilliary qubit to |-> (to give it a phase), and then sends both the 'phased' register (aux)
and the register-of-interest (x) to the phase kickback block.
"""
@qfunc
def zero_diffuzer(x: QNum):
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)

We have now come to the two subroutines that will execute the "walk" operatation. It is here that we will now see a deviation from the given example, specifically in the `W_iteration()` subroutine, as `prob` list variable is what gives our graph its structure. In the original example there are 4 nodes (vertices) oriented in a closed loop (that is, node 0 is connected to nodes 3 and 1). Here, with a linear chain, node 0 will only be connected to 1, and node 15 will only be connected to node 14. Put another way, they will only have one edge each. All other nodes (1 thru 14 inclusive) will be connected to two neighbors: plus 1 and minus 1 from themselves. That is, they will each have two edges.

I have opted to make this subroutine more generic for linear chains of any length by using the `SIZE` variable.

#### **IMPORTANT NOTE HERE!**
In the closed loop, each vertex was doubly connected--as such each 'step' in the walk could take either path. Therefore, the probabilities reflected this equally--with 0.5. HOWEVER, because the edge vertexes (0 and 15 in this case) only have one edge each, that means that they can only 'walk' along those edges to their adjacent nodes (1 and 14 respectively). Therefore, the probabilities are 1.0 exactly.

***Side note:*** Regardless of number of vertices or linear versus loop--one could expand this walk to include options for "staying sill" and not moving along any edge. The probabilities for a doubly connected vertex would then be in thirds (1/3 for moving down an integer; 1/3 for moving up an integer; 1/3 for 'current' position). The nodes at the ends of linear chains in this model would then be equal probabilities of 1/2 each (move or stay put).

In [18]:
"""
This function executes a single 'step' of our quantum walk for a given node/vertex i.
"""
def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    # Initialize an empty vector for each state.
    prob = [0] * SIZE**2
    modulus_value = SIZE**2

    # As this is for linear chains, the first and last nodes/vertices will only have
    # 1 connecting edge each. The 'internal' nodes will each have two and can be
    # connected in the same way as the 4 node ring example.
    if i == 0:
        prob[(i+1) % modulus_value] = 1
    elif i == (SIZE**2 - 1):
        prob[(i-1) % modulus_value] = 1
    else:
        prob[(i+1) % modulus_value] = 0.5
        prob[(i-1) % modulus_value] = 0.5

    print(f'State={i}, prob vec ={prob}')

    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))

#-----------------------------------------------------------------------------------------------------------

"""
This little subroutine enables us to iterate over each node/vertex in the graph and "walk" from it using
the W_iteration.
"""
@qfunc
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    for i in range(2**SIZE):
      W_iteration(i,vertices,adjacent_vertices)

The previous cells execute steps in our quantum walk.

In the next block we execute the other equation required for the algorithm. This is the SWAP stuff--that is, the population on a vertex is given by its probability amplitude. A SWAP gate indicates that we have walked somewhere--that is, if we were currently on vertex v and 'take a step' to vertex v+1, we are moving the state on v to v+1. This would manifest as a population difference if we take a measurement.

Overall the code doesn't really change, except for the all-important `edge_oracle()`. Here in this work, the nodes are labeled linearly, like this:  
0--1--2--3--...--13--14--15  

So, if we are currently on node 3, we can either move to the left (to 2) or to the right (4). Either way, we can determine that 3 is in fact sharing an edge with 2 and 4 because the sum of any two numbers will be even if the parity of addends is the same or odd if the addends' parity does not match.

HOWEVER, this alone is not sufficient to determine if an edge is shared. After all, node 3 and node 8 have opposite parities, but they are NOT connected! They do not share an edge! So the example given in class is insufficient.

Instead, we can ask the computer to compute the numerical difference between the variable `vertices` and `adjacent_vertices` to determine if they are next to each other. If they are, the difference will be $\pm1$. If they are not connected, the difference will be some other number. However, we don't know if the difference will be $+1$ (like if `vertices` is 5 and `adjacent_vertices` is 4) or $-1$ (ie, if `vertices` is 5 and `adjacent_vertices` is 6). By squaring the difference, we gauruntee that the result is positive semi-definite. Thus, if the squared difference is exactly 1, we know the nodes are adjacent. Any other value implies that the target vertices are not adjacent.

We note that this would not work for the loop version shown in class as the difference between the terminal nodes (0 and 3) which share an edge would have a squared difference of 9, and thus fail this test. But for any linear graph, this is how we proceed.

In [38]:
"""
This subroutine determines whether two nodes (vertices) are adjacent. If they are, return a 1. If they are not, return a 0.
This function is different from the one given in the example, according to the logic presented in the text above. Please
consult that information for the details!
"""
@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
    res |= (((vertices-adjacent_vertices)**2) ==1)

#-----------------------------------------------------------------------------------------------------------

"""
This function 'swaps' the states of two adjacent nodes when a 'step' is taken. This is explained with more
detail in the first paragraph of the previous 'text' block.
"""
@qfunc
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
  repeat(count= x.len,
    iteration= lambda i: SWAP(x[i],y[i]))

#-----------------------------------------------------------------------------------------------------------

"""
The subroutine below is the primitive setup to first determine whether two vertices share an edge, and if so,
when we 'take a step', that process is executed by swapping the states of two (adjacent) nodes.
"""
@qfunc
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

And now, someone to run the show--we need a `main()` and the necessary syntax to synthesize and then viewing it using the Classiq backend, which we call to. This code is identical to the example presented in class.

In [39]:
@qfunc
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):

  allocate(SIZE,vertices)
  hadamard_transform(vertices)
  allocate(SIZE,adjacent_vertices)

  W_operator(vertices,adjacent_vertices)
  S_operator(vertices,adjacent_vertices)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

State=0, prob vec =[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 